In [ ]:
!pip install -q git+https://github.com/openai/whisper.git tqdm
!pip install -U datasets huggingface_hub fsspec jiwer

In [ ]:
!pip install "numpy<2.0" --upgrade

# Install FFmpeg for audio processing (fixes channel issues)
!apt-get update -qq
!apt-get install -y ffmpeg

!pip install openai-whisper
!pip install datasets==3.5.0
!pip install torchcodec  # Required for audio decoding
# Alternative audio libraries if torchcodec fails
!pip install librosa soundfile

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.3.2
    Uninstalling numpy-2.3.2:
      Successfully uninstalled numpy-2.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.7.0 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.1 which is incompatible.
thinc 8.3.6 requires 

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 43 not upgraded.
  Using cached fsspec-2024.12.0-py3-none-any.whl.metadata (11 kB)
Using cached fsspec-2024.12.0-py3-none-any.whl (183 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.7.0
    Uninstalling fsspec-2025.7.0:
      Successfully uninstalled fsspec-2025.7.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
fastai 2.7.19 requires torch<2.7

In [ ]:
!pip install torch torchvision torchaudio --force-reinstall
!pip install torchcodec --force-reinstall


  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 129.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB

  Using cached torchcodec-0.5-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (10 kB)
Using cached torchcodec-0.5-cp311-cp311-manylinux_2_28_x86_64.whl (1.4 MB)
  Attempting uninstall: torchcodec
    Found existing installation: torchcodec 0.5
    Uninstalling torchcodec-0.5:
      Successfully uninstalled torchcodec-0.5


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch.nn import CrossEntropyLoss
from torch.optim.lr_scheduler import LinearLR
from torch.nn.utils.rnn import pad_sequence
import numpy as np
import whisper
from transformers import AutoTokenizer, AutoModel
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
import librosa
import warnings
from datasets import load_dataset
import pandas as pd
import time  # Added for timing training

# Import audio processing libraries
import torchaudio
import torchaudio.transforms as T
try:
    import torchcodec
    print("✅ torchcodec available")
except ImportError:
    print("ℹ️  torchcodec not available, using alternatives")

warnings.filterwarnings("ignore")

print("✅ All libraries imported successfully")
print(f"PyTorch version: {torch.__version__}")
print(f"Torchaudio version: {torchaudio.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

✅ torchcodec available
✅ All libraries imported successfully
PyTorch version: 2.7.1+cu126
Torchaudio version: 2.7.1+cu126
CUDA available: True


In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
# ===== CELL 4: Load Datasets (Memory Efficient Version) =====

# Load training/validation dataset
print("Loading training/validation dataset...")
ds = load_dataset("UBC-NLP/NADI2025_subtask2_SLID")
ds.set_format("torch")

print("Dataset structure:")
print(ds)

# Define dialects but DON'T load test data yet (memory efficient approach)
print("\nSetting up test dataset configuration...")
dialects = ['Algeria', 'Egypt', 'Jordan', 'Mauritania', 'Morocco', 'Palestine', 'UAE', 'Yemen']

# Instead of loading all data, we'll check dataset sizes first
print("Checking test dataset sizes...")
total_test_samples = 0
dialect_sizes = {}

for dialect in dialects:
    try:
        print(f"Checking {dialect}...")
        # Load just to get size info, then delete
        dialect_ds = load_dataset("UBC-NLP/NADI2025_subtask2_ASR_Test", dialect)
        size = len(dialect_ds['test'])
        dialect_sizes[dialect] = size
        total_test_samples += size
        print(f"  {dialect}: {size:,} samples")

        # Clear from memory immediately
        del dialect_ds

    except Exception as e:
        print(f"  ❌ Error checking {dialect}: {e}")
        dialect_sizes[dialect] = 0

print(f"\nTotal test samples across all dialects: {total_test_samples:,}")
print(f"Memory efficient approach: Will process each dialect separately during transcription")

# Store configuration for later use
TEST_CONFIG = {
    'dialects': dialects,
    'sizes': dialect_sizes,
    'total_samples': total_test_samples
}

print("✅ Dataset configuration complete - using memory-efficient processing")


Loading training/validation dataset...


README.md:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

train-00000-of-00004.parquet:   0%|          | 0.00/456M [00:00<?, ?B/s]

train-00001-of-00004.parquet:   0%|          | 0.00/416M [00:00<?, ?B/s]

train-00002-of-00004.parquet:   0%|          | 0.00/469M [00:00<?, ?B/s]

...

  UAE: 1,600 samples
Checking Yemen...


test-00000-of-00002.parquet:   0%|          | 0.00/364M [00:00<?, ?B/s]

test-00001-of-00002.parquet:   0%|          | 0.00/417M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/1183 [00:00<?, ? examples/s]

  Yemen: 1,183 samples

Total test samples across all dialects: 10,810
Memory efficient approach: Will process each dialect separately during transcription
✅ Dataset configuration complete - using memory-efficient processing


In [ ]:
# ===== CELL 5: Define Country Labels and Helper Functions =====

# Country labels for classification
countries = ['Algeria', 'Egypt', 'Jordan', 'Mauritania', 'Morocco', 'Palestine', 'UAE', 'Yemen']
labels2id = {key:id for id, key in enumerate(countries)}
id2labels = {id:key for key,id in labels2id.items()}

print("Country labels mapping:")
for i, country in enumerate(countries):
    print(f"{i}: {country}")

# Helper function to check trainable parameters
def print_trainable_parameters(model):
    """
    Print trainable parameters with detailed breakdown
    """
    trainable_params = 0
    all_param = 0

    print("\n=== MODEL PARAMETER BREAKDOWN ===")
    for name, param in model.named_parameters():
        param_count = param.numel()
        all_param += param_count
        if param.requires_grad:
            trainable_params += param_count
            print(f"✓ TRAINABLE: {name:40} - {param_count:>10,} params")
        else:
            print(f"✗ FROZEN:    {name:40} - {param_count:>10,} params")

    print(f"\n=== SUMMARY ===")
    print(f"Total parameters:     {all_param:>15,}")
    print(f"Trainable parameters: {trainable_params:>15,}")
    print(f"Frozen parameters:    {all_param - trainable_params:>15,}")
    print(f"Trainable percentage: {100 * trainable_params / all_param:>14.2f}%")
    print(f"Model size:           {all_param/1e6:>14.1f}M parameters")
    print(f"Training only:        {trainable_params/1e6:>14.1f}M parameters")

    # Memory efficiency info
    trainable_memory_mb = trainable_params * 4 / (1024 * 1024)  # Assuming float32
    print(f"Trainable memory:     {trainable_memory_mb:>14.1f}MB")
    print("=" * 50)

Country labels mapping:
0: Algeria
1: Egypt
2: Jordan
3: Mauritania
4: Morocco
5: Palestine
6: UAE
7: Yemen


In [ ]:
# ===== CELL 6: Define Cost Calculation Functions =====

def llr(logits):
    """Calculate log likelihood ratios"""
    classes = logits.shape[1]
    l2 = logits.unsqueeze(dim=1)

    l = logits.unsqueeze(dim=1)
    l = l.repeat(1,8,1)
    l2 = l2.repeat(1,8,1)
    l2 = l2.permute((0,2,1))
    dif = l-l2
    e = torch.exp(dif)
    for i in range(len(e)):
        e[i].fill_diagonal_(0)
    return -torch.log(torch.sum(e,dim=-1)/ (classes-1))

def compute_actual_cost(scores, labels, p_target, c_miss=1, c_fa=1):
    """Actual Cost Function from the NIST Speech Recognition Evaluation tool package"""
    beta = c_fa * (1 - p_target) / (c_miss * p_target)
    decisions = (scores >= np.log(beta)).astype('i')
    num_targets = np.sum(labels)
    fp = np.sum(decisions * (1 - labels))
    num_nontargets = np.sum(1 - labels)
    fn = np.sum((1 - decisions) * labels)
    fpr = fp / num_nontargets if num_nontargets > 0 else np.nan
    fnr = fn / num_targets if num_targets > 0 else np.nan
    return fnr + beta * fpr, fpr, fnr

def compute_ave_cost(logits, labels, num_l=8):
    """Compute average cost"""
    llratio = llr(logits)
    llratio = llratio.numpy()
    labels = labels.numpy()
    order = labels.argsort()
    labels.sort()
    llratio = llratio[order]
    indices = np.where(labels[:-1] != labels[1:])[0]
    indices = np.append(indices, [-1])
    one_hot = np.eye(8)[labels]
    fprs = []
    fnrs = []
    last = 0
    for i in indices:
        _, fpr, fnr = compute_actual_cost(llratio[last:i], one_hot[last:i], 0.5)
        fprs.append(fpr)
        fnrs.append(fnr)
        last = i
    fpr = sum(fprs)/num_l
    fnr = sum(fnrs)/num_l
    cost = fpr+fnr
    return cost, fpr, fnr

In [ ]:
# ===== CELL 7: Define Enhanced Collate Function with Torchaudio =====

def col_fun_whisper_enhanced(samples):
    """
    Enhanced collate function using torchaudio for better audio processing
    Works for both training (with labels) and test (without labels) data
    """
    arrays = []
    countries = []
    ids = []

    for sample in samples:
        # Get ID
        if 'ID' in sample:
            ids.append(sample['ID'])

        # Get audio array
        audio_array = sample['audio']['array']
        sample_rate = sample['audio'].get('sampling_rate', 16000)

        # Convert to tensor if needed
        if isinstance(audio_array, np.ndarray):
            audio_tensor = torch.from_numpy(audio_array).float()
        else:
            audio_tensor = audio_array.float()

        # Ensure mono audio
        if audio_tensor.dim() > 1:
            audio_tensor = torch.mean(audio_tensor, dim=0)

        # Resample to 16kHz if needed (Whisper expects 16kHz)
        if sample_rate != 16000:
            resampler = T.Resample(orig_freq=sample_rate, new_freq=16000)
            audio_tensor = resampler(audio_tensor)

        # Normalize audio
        if audio_tensor.abs().max() > 0:
            audio_tensor = audio_tensor / audio_tensor.abs().max()

        # Pad or truncate to reasonable length (30 seconds max for Whisper)
        max_length = 16000 * 30  # 30 seconds at 16kHz
        if len(audio_tensor) > max_length:
            audio_tensor = audio_tensor[:max_length]

        arrays.append(audio_tensor)

        # Get country label if available (for training data)
        if 'country' in sample:
            countries.append(labels2id[sample['country']])

    # Pad sequences to same length
    lengths = [len(arr) for arr in arrays]
    max_len = max(lengths)

    padded_arrays = []
    for arr in arrays:
        if len(arr) < max_len:
            padded = torch.nn.functional.pad(arr, (0, max_len - len(arr)))
        else:
            padded = arr
        padded_arrays.append(padded)

    packed = torch.stack(padded_arrays)

    if countries:  # Training data
        countries_tensor = torch.tensor(countries)
        return packed, lengths, countries_tensor, ids
    else:  # Test data
        return packed, lengths, None, ids

# Also keep the original collate function as backup
def col_fun_whisper(samples):
    """
    Original collate function (backup)
    """
    arrays = []
    countries = []
    ids = []

    for sample in samples:
        # Get ID
        if 'ID' in sample:
            ids.append(sample['ID'])

        # Get audio array
        audio_array = sample['audio']['array']

        # Ensure audio is in correct format for Whisper (16kHz, mono)
        if isinstance(audio_array, torch.Tensor):
            audio_array = audio_array.numpy()

        # Pad or truncate to reasonable length (30 seconds max for Whisper)
        max_length = 16000 * 30  # 30 seconds at 16kHz
        if len(audio_array) > max_length:
            audio_array = audio_array[:max_length]

        arrays.append(torch.from_numpy(audio_array).float())

        # Get country label if available (for training data)
        if 'country' in sample:
            countries.append(labels2id[sample['country']])

    # Pad sequences to same length
    lengths = [len(arr) for arr in arrays]
    max_len = max(lengths)

    padded_arrays = []
    for arr in arrays:
        if len(arr) < max_len:
            padded = torch.nn.functional.pad(arr, (0, max_len - len(arr)))
        else:
            padded = arr
        padded_arrays.append(padded)

    packed = torch.stack(padded_arrays)

    if countries:  # Training data
        countries_tensor = torch.tensor(countries)
        return packed, lengths, countries_tensor, ids
    else:  # Test data
        return packed, lengths, None, ids

print("✅ Enhanced collate function with torchaudio support defined!")
print("ℹ️  Will use enhanced version with audio preprocessing")

✅ Enhanced collate function with torchaudio support defined!
ℹ️  Will use enhanced version with audio preprocessing


In [ ]:
# ===== CELL 8: Define the Main Model Class =====

# Import required modules for the model class
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
import whisper

class WhisperARBERTDialectClassifier(nn.Module):
    """
    Combined Whisper ASR + ARBERT/MARBERT text classification pipeline
    Following efficient transfer learning approach like SpeechBrain
    """
    def __init__(self, whisper_model_name="large-v3", text_model_name="UBC-NLP/MARBERT", num_classes=8):
        super().__init__()

        # Load Whisper model for ASR
        print(f"Loading Whisper model: {whisper_model_name}")
        self.whisper_model = whisper.load_model(whisper_model_name)

        # Load ARBERT/MARBERT model for text classification
        print(f"Loading text model: {text_model_name}")
        self.tokenizer = AutoTokenizer.from_pretrained(text_model_name)

        # Load the base model without classification head (like SpeechBrain)
        self.text_encoder = AutoModel.from_pretrained(text_model_name)

        # Freeze Whisper model
        for param in self.whisper_model.parameters():
            param.requires_grad = False

        # Freeze text encoder (like SpeechBrain approach)
        for param in self.text_encoder.parameters():
            param.requires_grad = False

        # Replace classifier layer (similar to SpeechBrain approach)
        # Get hidden size from the text encoder
        hidden_size = self.text_encoder.config.hidden_size
        self.classifier = nn.Linear(hidden_size, num_classes)

        print(f"Created new classifier: {hidden_size} -> {num_classes}")
        print(f"Frozen Whisper parameters: {sum(p.numel() for p in self.whisper_model.parameters()):,}")
        print(f"Frozen text encoder parameters: {sum(p.numel() for p in self.text_encoder.parameters()):,}")
        print(f"Trainable classifier parameters: {sum(p.numel() for p in self.classifier.parameters()):,}")

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def transcribe_audio(self, audio_tensor):
        """
        Transcribe audio using Whisper
        """
        # Convert tensor to numpy and ensure proper format
        if isinstance(audio_tensor, torch.Tensor):
            audio_numpy = audio_tensor.cpu().numpy()
        else:
            audio_numpy = audio_tensor

        # Whisper expects audio in specific format
        if audio_numpy.ndim > 1:
            audio_numpy = audio_numpy.squeeze()

        # Transcribe with Whisper
        result = self.whisper_model.transcribe(
          audio_numpy,
          language='ar',
          task='transcribe',
          beam_size=1,    # Higher quality (default for v3)
          best_of=1,      # Better results
          temperature=0   # Deterministic
        )

        return result['text']

    def classify_text(self, text):
        """
        Classify text using frozen ARBERT/MARBERT encoder + new classifier
        """
        # Tokenize text
        inputs = self.tokenizer(
            text,
            return_tensors='pt',
            truncation=True,
            padding=True,
            max_length=512
        )

        # Move to device
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        # Get embeddings from frozen text encoder
        with torch.no_grad():
            encoder_outputs = self.text_encoder(**inputs)
            # Use [CLS] token embedding (first token)
            embeddings = encoder_outputs.last_hidden_state[:, 0, :]

        # Pass through trainable classifier
        logits = self.classifier(embeddings)

        return logits

    def forward(self, audio_batch, training=False, return_transcriptions=False):
        """
        Forward pass: Audio -> Whisper -> Text -> Frozen ARBERT -> New Classifier -> Logits
        """
        batch_size = audio_batch.shape[0]
        all_logits = []
        transcriptions = []

        # During training, we need to handle gradients properly
        if training:
            # Pre-transcribe all audio to avoid gradient issues
            transcripts = []
            for i in range(batch_size):
                audio_sample = audio_batch[i].cpu().numpy()
                try:
                    transcript = self.transcribe_audio(audio_sample)
                    if not transcript.strip():
                        transcript = "لا يوجد نص"  # "No text" in Arabic
                    transcripts.append(transcript)
                    if return_transcriptions:
                        transcriptions.append(transcript)
                except Exception as e:
                    print(f"Error transcribing audio {i}: {e}")
                    transcripts.append("لا يوجد نص")
                    if return_transcriptions:
                        transcriptions.append("لا يوجد نص")

            # Batch process all transcripts
            if transcripts:
                inputs = self.tokenizer(
                    transcripts,
                    return_tensors='pt',
                    truncation=True,
                    padding=True,
                    max_length=512
                )
                inputs = {k: v.to(self.device) for k, v in inputs.items()}

                # Get embeddings from frozen encoder
                with torch.no_grad():
                    encoder_outputs = self.text_encoder(**inputs)
                    embeddings = encoder_outputs.last_hidden_state[:, 0, :]

                # Pass through trainable classifier
                final_logits = self.classifier(embeddings)
            else:
                final_logits = torch.randn(batch_size, 8).to(self.device)

        else:
            # Inference mode - process individually
            for i in range(batch_size):
                audio_sample = audio_batch[i].cpu().numpy()

                try:
                    # Transcribe audio
                    transcript = self.transcribe_audio(audio_sample)

                    # Handle empty transcriptions
                    if not transcript.strip():
                        transcript = "لا يوجد نص"  # "No text" in Arabic

                    # Store transcription if requested
                    if return_transcriptions:
                        transcriptions.append(transcript)

                    # Classify text
                    logits = self.classify_text(transcript)
                    all_logits.append(logits)

                except Exception as e:
                    print(f"Error processing audio sample {i}: {e}")
                    # Return random logits as fallback
                    fallback_logits = torch.randn(1, 8).to(self.device)
                    all_logits.append(fallback_logits)
                    if return_transcriptions:
                        transcriptions.append("لا يوجد نص")

            # Concatenate all logits
            final_logits = torch.cat(all_logits, dim=0)

        # Get predictions
        predictions = torch.argmax(final_logits, dim=1)

        # Get max scores (for compatibility with baseline)
        max_scores = torch.max(final_logits, dim=1)[0]

        # Return in same format as baseline: (logits, max_scores, predictions, country_names)
        country_names = [id2labels[pred.item()] for pred in predictions]

        if return_transcriptions:
            return final_logits, max_scores, predictions, country_names, transcriptions
        else:
            return final_logits, max_scores, predictions, country_names

print("Model class defined successfully!")

Model class defined successfully!


In [ ]:
# ===== CELL 9: Create and Initialize Model =====

# Create the model
model = WhisperARBERTDialectClassifier(
    whisper_model_name="large-v3",  # Using Whisper Large v3
    text_model_name="UBC-NLP/MARBERT",  # or "UBC-NLP/ARBERT"
    num_classes=8
)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Loading Whisper model: large-v3


100%|█████████████████████████████████████| 2.88G/2.88G [02:08<00:00, 24.1MiB/s]


Loading text model: UBC-NLP/MARBERT


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Created new classifier: 768 -> 8
Frozen Whisper parameters: 1,541,570,560
Frozen text encoder parameters: 162,841,344
Trainable classifier parameters: 6,152
Using device: cuda


In [ ]:
# ===== CELL 10: Create Data Loaders (Enhanced with Validation Training) =====

# Check if datasets were loaded successfully
if 'ds' not in locals() and 'ds' not in globals():
    print("❌ Training/validation dataset not found. Loading now...")
    try:
        ds = load_dataset("UBC-NLP/NADI2025_subtask2_SLID")
        ds.set_format("torch")
        print("✅ Training/validation dataset loaded successfully")
    except Exception as e:
        print(f"❌ Failed to load training/validation dataset: {e}")
        raise e

# Try enhanced collate function first, fall back to original if needed
try:
    print("🔄 Testing enhanced collate function with torchaudio...")
    test_sample = [ds['train'][0]]  # Test with one sample
    col_fun_whisper_enhanced(test_sample)
    collate_fn_to_use = col_fun_whisper_enhanced
    print("✅ Using enhanced collate function with torchaudio preprocessing")
except Exception as e:
    print(f"⚠️  Enhanced collate function failed: {e}")
    print("🔄 Falling back to original collate function...")
    collate_fn_to_use = col_fun_whisper
    print("✅ Using original collate function")

# OPTION 1: Standard training (train only on training set)
print("\n🤔 Training Data Configuration Options:")
print("1️⃣  Standard: Use only training data (12,900 samples)")
print("2️⃣  Enhanced: Use training + validation data (25,600 samples)")

# Let user choose or default to enhanced
USE_VALIDATION_FOR_TRAINING = True  # Set to False if you want standard training only

if USE_VALIDATION_FOR_TRAINING:
    print("✅ Selected: Enhanced training (using validation data for training)")

    # Combine training and validation datasets
    from datasets import concatenate_datasets
    combined_train_ds = concatenate_datasets([ds['train'], ds['validation']])

    # Create enhanced training loader with combined data
    trainloader = DataLoader(
        combined_train_ds,
        shuffle=True,
        collate_fn=collate_fn_to_use,
        batch_size=8
    )

    # For evaluation, we'll use a subset of the validation data (or skip evaluation)
    # Option A: Use small subset of validation for evaluation
    val_subset_size = 10  # Use only 1000 samples for quick evaluation
    val_indices = list(range(0, min(val_subset_size, len(ds['validation']))))
    val_subset = ds['validation'].select(val_indices)

    valloader = DataLoader(
        val_subset,
        shuffle=False,
        collate_fn=collate_fn_to_use,
        batch_size=4
    )

    print(f"📊 Enhanced training configuration:")
    print(f"   🎯 Training samples: {len(combined_train_ds):,} (train + validation)")
    print(f"   📊 Evaluation samples: {len(val_subset):,} (subset of validation)")
    print(f"   🚀 More data = potentially better performance!")

else:
    print("✅ Selected: Standard training (using only training data)")

    # Standard configuration
    trainloader = DataLoader(ds['train'], shuffle=True, collate_fn=collate_fn_to_use, batch_size=8)
    valloader = DataLoader(ds['validation'], shuffle=False, collate_fn=collate_fn_to_use, batch_size=4)

    print(f"📊 Standard training configuration:")
    print(f"   🎯 Training samples: {len(ds['train']):,}")
    print(f"   📊 Evaluation samples: {len(ds['validation']):,}")

# No test loader creation here - we'll process dialects individually
print("ℹ️  Test data will be processed dialect-by-dialect to save memory")

print("\nData loaders summary:")
print(f"✅ Training batches: {len(trainloader):,}")
print(f"✅ Evaluation batches: {len(valloader):,}")
print(f"🎵 Audio processing: {'Enhanced (torchaudio)' if collate_fn_to_use == col_fun_whisper_enhanced else 'Standard'}")
print(f"📈 Training approach: {'Enhanced (train+val)' if USE_VALIDATION_FOR_TRAINING else 'Standard (train only)'}")
print(f"ℹ️  Test processing: Memory-efficient dialect-by-dialect approach")

# Store configuration for later use
if 'TEST_CONFIG' in locals() or 'TEST_CONFIG' in globals():
    DIALECTS = TEST_CONFIG['dialects']
    print(f"ℹ️  Will process {TEST_CONFIG['total_samples']:,} test samples across {len(DIALECTS)} dialects")
else:
    DIALECTS = ['Algeria', 'Egypt', 'Jordan', 'Mauritania', 'Morocco', 'Palestine', 'UAE', 'Yemen']
    print(f"ℹ️  Using default dialects list: {DIALECTS}")

# Store the collate function for later use
COLLATE_FN = collate_fn_to_use

# Force garbage collection to free up memory
import gc
gc.collect()
print("🧹 Memory cleanup completed")

🔄 Testing enhanced collate function with torchaudio...


model.safetensors:   0%|          | 0.00/654M [00:00<?, ?B/s]

✅ Using enhanced collate function with torchaudio preprocessing

🤔 Training Data Configuration Options:
1️⃣  Standard: Use only training data (12,900 samples)
2️⃣  Enhanced: Use training + validation data (25,600 samples)
✅ Selected: Enhanced training (using validation data for training)
📊 Enhanced training configuration:
   🎯 Training samples: 25,600 (train + validation)
   📊 Evaluation samples: 10 (subset of validation)
   🚀 More data = potentially better performance!
ℹ️  Test data will be processed dialect-by-dialect to save memory

Data loaders summary:
✅ Training batches: 3,200
✅ Evaluation batches: 3
🎵 Audio processing: Enhanced (torchaudio)
📈 Training approach: Enhanced (train+val)
ℹ️  Test processing: Memory-efficient dialect-by-dialect approach
ℹ️  Will process 10,810 test samples across 8 dialects
🧹 Memory cleanup completed


In [ ]:
# ===== CELL 11: Define Training Function (Fast Version with Enhanced Data) =====

def train_whisper_arbert_model_fast(model, trainloader, valloader, device, skip_eval=False):
    """
    Fast training loop for Whisper + ARBERT model
    Can optionally skip evaluation entirely for maximum speed
    """
    model = model.to(device)
    model.device = device

    # Only train the new classifier layer (like SpeechBrain approach)
    optimizer = AdamW(model.classifier.parameters(), lr=2e-5)

    max_steps = 5000 if len(trainloader) > 2000 else 3000  # More steps if more data
    logging_steps = 250  # Adjusted for potentially longer training

    print(f"📊 Training configuration:")
    print(f"   🎯 Total steps: {max_steps:,}")
    print(f"   📈 Training samples per epoch: ~{len(trainloader) * 8:,}")
    print(f"   ⚡ Final evaluation only: {'No evaluation' if skip_eval else 'Yes'}")

    # Simple learning rate schedule
    scheduler = LinearLR(optimizer, start_factor=0.1, end_factor=1.0, total_iters=300)

    loss_fn = CrossEntropyLoss()

    avg_loss = 0
    step = 1

    print("🚀 Starting FAST training with enhanced data...")
    print_trainable_parameters(model)

    model.train()

    start_time = time.time()

    while step <= max_steps:

        for batch in tqdm(trainloader, desc=f"Training Step {step}", leave=False):
            if step > max_steps:
                break

            wavs, _, labels, _ = batch  # Updated to handle IDs
            labels = labels.to(device)

            optimizer.zero_grad()

            try:
                # Forward pass with training flag
                o = model(wavs, training=True)
                logits = o[0]

                # Calculate loss
                loss = loss_fn(logits, labels)
                loss.backward()

                optimizer.step()
                if step <= 300:
                    scheduler.step()

                avg_loss += loss.item()

                # Logging (reduced frequency)
                if step % logging_steps == 0:
                    elapsed_time = time.time() - start_time
                    steps_per_sec = step / elapsed_time
                    eta_seconds = (max_steps - step) / steps_per_sec
                    eta_minutes = eta_seconds / 60

                    print(f"Step {step}/{max_steps}: Loss {avg_loss/logging_steps:.4f}, "
                          f"LR {optimizer.param_groups[0]['lr']:.2e}, "
                          f"Speed: {steps_per_sec:.1f} steps/s, "
                          f"ETA: {eta_minutes:.1f}min")
                    avg_loss = 0

                step += 1

            except Exception as e:
                print(f"Error in training step {step}: {e}")
                # Skip this batch and continue
                step += 1
                continue

    total_time = time.time() - start_time
    print(f"⏱️  Total training time: {total_time/3600:.1f} hours ({total_time/60:.1f} minutes)")

    # Evaluate only if not skipping
    if not skip_eval and valloader is not None:
        print(f"\n📊 Training completed! Running final evaluation...")
        final_acc, final_cost, final_fpr, final_fnr = eval_loop_whisper(model, valloader)
        print(f"🎯 Final Results: ACC {final_acc:.2f}% AvgCost {final_cost:.4f}, FPR {final_fpr:.4f}, FNR {final_fnr:.4f}")
    else:
        print(f"✅ Training completed! (Evaluation skipped for maximum speed)")

    return model

print("⚡ Enhanced fast training function defined successfully!")
print("💡 Features: Optional evaluation skip, adaptive training steps, enhanced data support")


⚡ Enhanced fast training function defined successfully!
💡 Features: Optional evaluation skip, adaptive training steps, enhanced data support


In [ ]:
# ===== CELL 12: Define Evaluation Function =====

def eval_loop_whisper(model, loader):
    """
    Evaluation loop for Whisper + ARBERT model
    """
    model.eval()
    tp = 0
    n = 0
    logits_list = []
    labels_list = []

    with torch.no_grad():
        for batch in tqdm(loader, desc="Evaluating"):
            wavs, _, labels, _ = batch  # Updated to handle IDs

            try:
                # Forward pass in evaluation mode
                o = model(wavs, training=False)

                # Collect results
                logits_list.append(o[0].cpu())
                labels_list.append(labels.cpu())

                # Calculate accuracy
                preds = o[2].cpu()
                tp += accuracy_score(labels, preds, normalize=False)
                n += len(labels)

            except Exception as e:
                print(f"Error in evaluation: {e}")
                # Skip this batch
                continue

    if len(logits_list) == 0:
        print("No successful predictions during evaluation!")
        return 0.0, 1.0, 1.0, 1.0

    # Concatenate results
    logits = torch.concat(logits_list, dim=0)
    labels = torch.concat(labels_list, dim=0)

    # Calculate cost metrics
    cost, fpr, fnr = compute_ave_cost(logits, labels)

    return tp/n * 100, cost, fpr, fnr

print("Evaluation function defined successfully!")

Evaluation function defined successfully!


In [ ]:
# ===== CELL 13: Define Transcription Function for Test Data =====

def transcribe_test_data(model, test_loader, output_file='test_transcriptions.csv'):
    """
    Transcribe test data and save results to CSV
    """
    model.eval()

    all_ids = []
    all_transcriptions = []
    all_predictions = []
    all_logits = []
    all_dialects = []  # Track original dialect labels

    print("Transcribing test data...")

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Processing test data"):
            wavs, _, _, ids = batch

            # Get dialect info from the original samples if available
            batch_dialects = []
            for sample_id in ids:
                # Try to find the dialect from the loaded data
                dialect_found = "Unknown"
                for sample in all_test_data:
                    if sample.get('ID') == sample_id:
                        dialect_found = sample.get('dialect', 'Unknown')
                        break
                batch_dialects.append(dialect_found)

            try:
                # Forward pass with transcription return enabled
                o = model(wavs, training=False, return_transcriptions=True)
                logits, max_scores, predictions, country_names, transcriptions = o

                # Store results
                all_ids.extend(ids)
                all_transcriptions.extend(transcriptions)
                all_predictions.extend(country_names)
                all_logits.extend(logits.cpu().numpy())
                all_dialects.extend(batch_dialects)

            except Exception as e:
                print(f"Error processing batch: {e}")
                # Handle batch failure
                batch_size = len(ids)
                all_ids.extend(ids)
                all_transcriptions.extend(["لا يوجد نص"] * batch_size)
                all_predictions.extend(["Egypt"] * batch_size)  # Default prediction
                all_logits.extend([np.random.randn(8)] * batch_size)
                all_dialects.extend(batch_dialects)

    # Create DataFrame with results
    results_df = pd.DataFrame({
        'ID': all_ids,
        'original_dialect': all_dialects,
        'transcription': all_transcriptions,
        'predicted_country': all_predictions
    })

    # Save transcriptions
    results_df.to_csv(output_file, index=False)
    print(f"Transcriptions saved to {output_file}")

    return results_df, all_logits

print("Transcription function defined successfully!")

Transcription function defined successfully!


In [ ]:
# ===== CELL 14: Define Submission Writing Functions =====

def write_logits(o, filename):
    with open(filename, "a") as fp:
        for i in o:
            l = ""
            for j in i:
                l += str(j.item()) + "\t"
            fp.write(str.strip(l)+"\n")

def write_preds(o, filename):
    with open(filename, "a") as fp:
        for i in o:
            fp.write(str.strip(str(i.item()))+"\n")

def submission_writer_whisper(model, loader, logits_file='logits.tsv', preds_file='predictions.tsv'):
    """
    Generate submission files for Whisper + ARBERT model
    Works with validation loader (has labels)
    """
    model.eval()

    # Clear existing files
    open(logits_file, 'w').close()
    open(preds_file, 'w').close()

    with torch.no_grad():
        for batch in tqdm(loader, desc="Generating submission"):
            wavs, _, labels, _ = batch

            # Forward pass
            o = model(wavs, training=False)

            # Write results
            write_logits(o[0].cpu(), logits_file)
            write_preds(o[2].cpu(), preds_file)

    print(f"✅ Submission files created:")
    print(f"   📄 {logits_file}")
    print(f"   📄 {preds_file}")

print("Submission functions defined successfully!")

Submission functions defined successfully!


# New Techinque: Only test data transcribe

In [ ]:
# ===== COMPLETE BATCH TRANSCRIPTION SYSTEM (NO TRAINING) =====
# Standalone system with all required components

import torch
import torch.nn as nn
import pandas as pd
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
import whisper
import numpy as np
from tqdm import tqdm
import gc
import time
import warnings
warnings.filterwarnings("ignore")

print("🚀 BATCH TRANSCRIPTION MODE - NO TRAINING")
print("="*60)
print("⚡ Optimized for speed with 10,000 test samples")
print("🎯 Using pretrained Whisper + simple classification")

# ===== STEP 1: Define Required Components =====

# Country labels mapping
countries = ['Algeria', 'Egypt', 'Jordan', 'Mauritania', 'Morocco', 'Palestine', 'UAE', 'Yemen']
labels2id = {key: id for id, key in enumerate(countries)}
id2labels = {id: key for key, id in labels2id.items()}

print("Country labels mapping:")
for i, country in enumerate(countries):
    print(f"  {i}: {country}")

# Enhanced collate function with error handling
def col_fun_whisper_batch(samples):
    """
    Robust collate function for batch processing with error handling
    """
    arrays = []
    ids = []

    print(f"    🔍 Processing batch with {len(samples)} samples")

    for i, sample in enumerate(samples):
        try:
            # Get ID
            sample_id = sample.get('ID', f'unknown_{i}')
            ids.append(sample_id)

            # Get audio array
            if 'audio' not in sample:
                print(f"    ⚠️  No audio in sample {sample_id}")
                # Create dummy audio
                arrays.append(torch.zeros(16000))  # 1 second of silence
                continue

            audio_data = sample['audio']
            if isinstance(audio_data, dict):
                audio_array = audio_data.get('array', [])
                sample_rate = audio_data.get('sampling_rate', 16000)
            else:
                audio_array = audio_data
                sample_rate = 16000

            # Handle empty or invalid audio
            if audio_array is None or len(audio_array) == 0:
                print(f"    ⚠️  Empty audio for sample {sample_id}")
                arrays.append(torch.zeros(16000))  # 1 second of silence
                continue

            # Convert to tensor if needed
            if isinstance(audio_array, np.ndarray):
                audio_tensor = torch.from_numpy(audio_array).float()
            else:
                audio_tensor = torch.tensor(audio_array).float()

            # Handle edge cases
            if audio_tensor.numel() == 0:
                print(f"    ⚠️  Zero-size tensor for sample {sample_id}")
                arrays.append(torch.zeros(16000))
                continue

            # Ensure mono audio
            if audio_tensor.dim() > 1:
                audio_tensor = torch.mean(audio_tensor, dim=0)

            # Ensure 1D tensor
            if audio_tensor.dim() == 0:
                audio_tensor = audio_tensor.unsqueeze(0)

            # Normalize audio safely
            max_val = audio_tensor.abs().max()
            if max_val > 0:
                audio_tensor = audio_tensor / max_val

            # Ensure minimum length
            min_length = 1600  # 0.1 seconds minimum
            if len(audio_tensor) < min_length:
                audio_tensor = torch.nn.functional.pad(audio_tensor, (0, min_length - len(audio_tensor)))

            # Truncate to reasonable length (30 seconds max for Whisper)
            max_length = 16000 * 30  # 30 seconds at 16kHz
            if len(audio_tensor) > max_length:
                audio_tensor = audio_tensor[:max_length]

            arrays.append(audio_tensor)

        except Exception as e:
            print(f"    ❌ Error processing sample {i}: {e}")
            # Add dummy data for failed sample
            ids.append(f'failed_{i}')
            arrays.append(torch.zeros(16000))

    # Handle empty batch
    if len(arrays) == 0:
        print(f"    ⚠️  Empty batch, creating dummy data")
        arrays = [torch.zeros(16000)]
        ids = ['dummy_0']

    # Pad sequences to same length
    lengths = [len(arr) for arr in arrays]
    if len(lengths) == 0:
        max_len = 16000
    else:
        max_len = max(lengths)

    padded_arrays = []
    for arr in arrays:
        if len(arr) < max_len:
            padded = torch.nn.functional.pad(arr, (0, max_len - len(arr)))
        else:
            padded = arr
        padded_arrays.append(padded)

    try:
        packed = torch.stack(padded_arrays)
    except Exception as e:
        print(f"    ❌ Error stacking tensors: {e}")
        # Create fallback tensor
        packed = torch.zeros(len(padded_arrays), max_len)
        for i, arr in enumerate(padded_arrays):
            packed[i, :len(arr)] = arr[:max_len]

    print(f"    ✅ Batch processed: {packed.shape}")

    # Return format: audio, lengths, labels (None for test), ids
    return packed, lengths, None, ids

# ===== STEP 2: Define Model Class =====

class WhisperARBERTDialectClassifier(nn.Module):
    """
    Batch-optimized Whisper + ARBERT model for transcription
    """
    def __init__(self, whisper_model_name="large-v3", text_model_name="UBC-NLP/MARBERT", num_classes=8):
        super().__init__()

        print(f"Loading Whisper model: {whisper_model_name}")
        self.whisper_model = whisper.load_model(whisper_model_name)

        print(f"Loading text model: {text_model_name}")
        self.tokenizer = AutoTokenizer.from_pretrained(text_model_name)
        self.text_encoder = AutoModel.from_pretrained(text_model_name)

        # Freeze all pretrained models
        for param in self.whisper_model.parameters():
            param.requires_grad = False
        for param in self.text_encoder.parameters():
            param.requires_grad = False

        # Simple classifier
        hidden_size = self.text_encoder.config.hidden_size
        self.classifier = nn.Linear(hidden_size, num_classes)

        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        print(f"✅ Model initialized - Whisper + MARBERT + Classifier")

    def transcribe_batch(self, audio_batch):
        """
        Efficiently transcribe a batch of audio samples
        """
        transcriptions = []

        for i in range(audio_batch.shape[0]):
            audio_numpy = audio_batch[i].cpu().numpy()

            try:
                # Whisper transcription
                result = self.whisper_model.transcribe(
                    audio_numpy,
                    language='ar',
                    task='transcribe',
                    beam_size=1,
                    best_of=1,
                    temperature=0
                )

                text = result['text'].strip()
                if not text:
                    text = "لا يوجد نص"

                transcriptions.append(text)

            except Exception as e:
                print(f"  ⚠️  Transcription error for sample {i}: {e}")
                transcriptions.append("لا يوجد نص")

        return transcriptions

    def classify_batch(self, texts):
        """
        Classify a batch of texts efficiently
        """
        if not texts:
            return torch.randn(1, 8).to(self.device)

        try:
            # Tokenize all texts at once
            inputs = self.tokenizer(
                texts,
                return_tensors='pt',
                truncation=True,
                padding=True,
                max_length=512
            )

            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # Get embeddings
            with torch.no_grad():
                encoder_outputs = self.text_encoder(**inputs)
                embeddings = encoder_outputs.last_hidden_state[:, 0, :]

            # Classify
            logits = self.classifier(embeddings)
            return logits

        except Exception as e:
            print(f"  ⚠️  Classification error: {e}")
            batch_size = len(texts)
            return torch.randn(batch_size, 8).to(self.device)

    def forward(self, audio_batch, return_transcriptions=False):
        """
        Complete forward pass: Audio -> Transcription -> Classification
        """
        # Transcribe all audio
        transcriptions = self.transcribe_batch(audio_batch)

        # Classify all transcriptions
        logits = self.classify_batch(transcriptions)

        # Get predictions and scores
        predictions = torch.argmax(logits, dim=1)
        max_scores = torch.max(logits, dim=1)[0]
        country_names = [id2labels[pred.item()] for pred in predictions]

        if return_transcriptions:
            return logits, max_scores, predictions, country_names, transcriptions
        else:
            return logits, max_scores, predictions, country_names

# ===== STEP 3: Initialize Model =====

print("\n🔧 Initializing model...")
model = WhisperARBERTDialectClassifier(
    whisper_model_name="large-v3",
    text_model_name="UBC-NLP/MARBERT",
    num_classes=8
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model.device = device

# Initialize classifier to predict Egypt (most common)
with torch.no_grad():
    model.classifier.weight.fill_(0.0)
    model.classifier.bias.fill_(0.0)
    model.classifier.bias[1] = 2.0  # Bias toward Egypt (index 1)

print("✅ Model ready for batch transcription!")

# ===== STEP 4: Batch Processing Function =====

def process_test_dialect_batch(dialect_name, batch_size=1):
    """
    Process one dialect with robust error handling
    """
    print(f"\n📂 Processing {dialect_name}...")

    try:
        # Load dialect dataset
        print(f"   🔄 Loading dataset for {dialect_name}...")
        dialect_ds = load_dataset("UBC-NLP/NADI2025_subtask2_ASR_Test", dialect_name)
        test_data = dialect_ds['test']

        print(f"   📊 {len(test_data):,} samples loaded")

        # Inspect first sample
        if len(test_data) > 0:
            sample = test_data[0]
            print(f"   🔍 Sample keys: {list(sample.keys())}")
            if 'audio' in sample:
                audio_info = sample['audio']
                if isinstance(audio_info, dict):
                    print(f"   🎵 Audio keys: {list(audio_info.keys())}")
                    if 'array' in audio_info:
                        array_shape = np.array(audio_info['array']).shape if audio_info['array'] is not None else "None"
                        print(f"   📐 Audio shape: {array_shape}")

        # Create data loader with error handling
        test_loader = DataLoader(
            test_data,
            batch_size=batch_size,
            shuffle=False,
            collate_fn=col_fun_whisper_batch,
            num_workers=0,
            pin_memory=False,
            drop_last=False  # Don't drop incomplete batches
        )

        print(f"   📦 Created data loader with {len(test_loader)} batches")

        # Process all batches
        all_results = []
        successful_batches = 0
        failed_batches = 0

        model.eval()
        with torch.no_grad():
            for batch_idx, batch in enumerate(tqdm(test_loader, desc=f"{dialect_name}")):
                try:
                    wavs, lengths, _, ids = batch

                    # Check batch validity
                    if wavs.numel() == 0:
                        print(f"   ⚠️  Empty batch {batch_idx}, skipping")
                        failed_batches += 1
                        continue

                    print(f"   🔄 Processing batch {batch_idx}: {wavs.shape} with {len(ids)} IDs")

                    # Process batch
                    results = model(wavs, return_transcriptions=True)
                    logits, max_scores, predictions, country_names, transcriptions = results

                    # Validate results
                    if len(transcriptions) != len(ids):
                        print(f"   ⚠️  Mismatch: {len(transcriptions)} transcriptions vs {len(ids)} IDs")

                    # Store results
                    for i in range(min(len(ids), len(transcriptions), len(country_names))):
                        all_results.append({
                            'ID': ids[i],
                            'dialect': dialect_name,
                            'transcription': transcriptions[i] if i < len(transcriptions) else 'لا يوجد نص',
                            'predicted_country': country_names[i] if i < len(country_names) else 'Egypt',
                            'prediction_id': predictions[i].cpu().item() if i < len(predictions) else 1,
                            'logits': logits[i].cpu().numpy() if i < len(logits) else np.random.randn(8),
                            'confidence': max_scores[i].cpu().item() if i < len(max_scores) else 0.5
                        })

                    successful_batches += 1

                except Exception as e:
                    print(f"   ❌ Batch {batch_idx} error: {e}")
                    failed_batches += 1

                    # Handle failed batch - create dummy results for all IDs in batch
                    try:
                        wavs, lengths, _, ids = batch
                        for i, sample_id in enumerate(ids):
                            all_results.append({
                                'ID': sample_id,
                                'dialect': dialect_name,
                                'transcription': 'لا يوجد نص',
                                'predicted_country': 'Egypt',
                                'prediction_id': 1,
                                'logits': np.random.randn(8),
                                'confidence': 0.5
                            })
                    except Exception as e2:
                        print(f"   ❌ Could not extract IDs from failed batch: {e2}")

                # Memory cleanup every 3 batches
                if batch_idx % 3 == 0 and torch.cuda.is_available():
                    torch.cuda.empty_cache()

        print(f"   ✅ Completed: {len(all_results)} transcriptions")
        print(f"   📊 Successful batches: {successful_batches}, Failed batches: {failed_batches}")

        # Cleanup
        del dialect_ds, test_data, test_loader
        gc.collect()

        return all_results

    except Exception as e:
        print(f"   ❌ Critical error processing {dialect_name}: {e}")
        import traceback
        traceback.print_exc()
        return []

# ===== STEP 5: Process All Dialects =====

dialects = ['Algeria', 'Egypt', 'Jordan', 'Mauritania', 'Morocco', 'Palestine', 'UAE', 'Yemen']
all_transcription_results = []

print(f"\n🔄 Processing {len(dialects)} dialects...")
start_time = time.time()

for i, dialect in enumerate(dialects):
    print(f"\n[{i+1}/{len(dialects)}] Starting {dialect}...")

    try:
        dialect_results = process_test_dialect_batch(dialect, batch_size=1)  # Reduced batch size

        if len(dialect_results) > 0:
            all_transcription_results.extend(dialect_results)
            print(f"   ✅ Added {len(dialect_results)} results from {dialect}")
        else:
            print(f"   ⚠️  No results from {dialect}")

        # Progress update
        elapsed = time.time() - start_time
        remaining_dialects = len(dialects) - (i + 1)

        if i > 0:  # Avoid division by zero
            avg_time_per_dialect = elapsed / (i + 1)
            eta = remaining_dialects * avg_time_per_dialect
            print(f"   ⏱️  Elapsed: {elapsed/60:.1f}min, ETA: {eta/60:.1f}min")

        print(f"   📊 Total results so far: {len(all_transcription_results):,}")

        # Force memory cleanup after each dialect
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    except Exception as e:
        print(f"   ❌ Critical error with {dialect}: {e}")
        import traceback
        traceback.print_exc()
        continue  # Try next dialect

total_time = time.time() - start_time

print(f"\n🎉 ALL DIALECTS COMPLETED!")
print(f"⏱️  Total time: {total_time/60:.1f} minutes")
print(f"📊 Total transcriptions: {len(all_transcription_results):,}")

# ===== STEP 6: Create Submission Files =====

print("\n📝 Creating submission files...")

# Convert to DataFrame
results_df = pd.DataFrame(all_transcription_results)

# Save detailed results
results_df.to_csv('batch_transcriptions.csv', index=False, encoding='utf-8')
print("✅ batch_transcriptions.csv saved")

# Create submission files
logits_array = np.stack(results_df['logits'].values)
predictions = results_df['prediction_id'].values

# Write logits file
with open('test_logits.tsv', 'w') as f:
    for logit_row in logits_array:
        f.write('\t'.join([f"{x:.6f}" for x in logit_row]) + '\n')

# Write predictions file
with open('test_predictions.tsv', 'w') as f:
    for pred in predictions:
        f.write(str(pred) + '\n')

print("✅ test_logits.tsv saved")
print("✅ test_predictions.tsv saved")

# Create submission zip
import zipfile
with zipfile.ZipFile('test_submission.zip', 'w') as zipf:
    zipf.write('test_logits.tsv')
    zipf.write('test_predictions.tsv')

print("✅ test_submission.zip created")

# ===== STEP 7: Final Summary =====

print("\n📊 BATCH PROCESSING SUMMARY")
print("="*60)
print(f"⏱️  Total processing time: {total_time/60:.1f} minutes")
print(f"📊 Total samples processed: {len(all_transcription_results):,}")
print(f"⚡ Average speed: {len(all_transcription_results)/(total_time/60):.1f} samples/minute")

# Show distribution
if len(results_df) > 0:
    country_dist = results_df['predicted_country'].value_counts()
    print(f"\n🗺️  Predicted Country Distribution:")
    for country, count in country_dist.items():
        percentage = (count / len(results_df)) * 100
        print(f"   {country}: {count:,} ({percentage:.1f}%)")

    # Show sample transcriptions
    print(f"\n📝 Sample Transcriptions:")
    for i in range(min(3, len(results_df))):
        row = results_df.iloc[i]
        print(f"   ID: {row['ID']}")
        print(f"   Dialect: {row['dialect']}")
        print(f"   Text: {row['transcription'][:80]}...")
        print(f"   Country: {row['predicted_country']}")
        print()

print("="*60)
print("🎉 BATCH TRANSCRIPTION COMPLETED!")
print("📦 Submit test_submission.zip to the competition")
print("="*60)

# Memory usage info
if torch.cuda.is_available():
    print(f"\n💾 Final GPU Memory Usage:")
    print(f"   Allocated: {torch.cuda.memory_allocated()/1024**3:.1f}GB")
    print(f"   Cached: {torch.cuda.memory_reserved()/1024**3:.1f}GB")

🚀 BATCH TRANSCRIPTION MODE - NO TRAINING
⚡ Optimized for speed with 10,000 test samples
🎯 Using pretrained Whisper + simple classification
Country labels mapping:
  0: Algeria
  1: Egypt
  2: Jordan
  3: Mauritania
  4: Morocco
  5: Palestine
  6: UAE
  7: Yemen

🔧 Initializing model...
Loading Whisper model: large-v3
Loading text model: UBC-NLP/MARBERT
✅ Model initialized - Whisper + MARBERT + Classifier
✅ Model ready for batch transcription!

🔄 Processing 8 dialects...

[1/8] Starting Algeria...

📂 Processing Algeria...
   🔄 Loading dataset for Algeria...
   📊 727 samples loaded
   🔍 Sample keys: ['audio', 'ID', 'duration']
   🎵 Audio keys: ['path', 'array', 'sampling_rate']
   📐 Audio shape: (31929,)
   📦 Created data loader with 727 batches


Algeria:   0%|          | 0/727 [00:00<?, ?it/s]

    🔍 Processing batch with 1 samples
    ✅ Batch processed: torch.Size([1, 31929])
   🔄 Processing batch 0: torch.Size([1, 31929]) with 1 IDs


Algeria:   0%|          | 1/727 [00:01<16:17,  1.35s/it]

    🔍 Processing batch with 1 samples
    ✅ Batch processed: torch.Size([1, 24937])
   🔄 Processing batch 1: torch.Size([1, 24937]) with 1 IDs


...

    🔍 Processing batch with 1 samples
    ✅ Batch processed: torch.Size([1, 264872])
   🔄 Processing batch 547: torch.Size([1, 264872]) with 1 IDs


Egypt:  34%|███▍      | 548/1600 [16:47<1:09:30,  3.96s/it]

    🔍 Processing batch with 1 samples
    ✅ Batch processed: torch.Size([1, 155755])
   🔄 Processing batch 548: torch.Size([1, 155755]) with 1 IDs


Egypt:  34%|███▍      | 549/1600 [16:49<55:54,  3.19s/it]  

    🔍 Processing batch with 1 samples
    ✅ Batch processed: torch.Size([1, 194931])
   🔄 Processing batch 549: torch.Size([1, 194931]) with 1 IDs


In [2]:
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoProcessor
import torch

model = AutoModelForCausalLM.from_pretrained("nvidia/audio-flamingo-3", torch_dtype=torch.float16).to("cuda")
processor = AutoProcessor.from_pretrained("nvidia/audio-flamingo-3")

def transcribe(files):
    inputs = processor(text="Transcribe in Arabic", audios=files, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=512)
    return processor.batch_decode(outputs, skip_special_tokens=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ValueError: The checkpoint you are trying to load has model type `llava_llama` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

In [2]:
!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers.git


Found existing installation: transformers 4.54.0
Uninstalling transformers-4.54.0:
  Successfully uninstalled transformers-4.54.0
  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-0lqosaly
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-0lqosaly
  Resolved https://github.com/huggingface/transformers.git to commit 1ad216bd7dced23e7e38ab4881a9528486588bd0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.55.0.dev0-py3-none-any.whl size=12116892 sha256=2d7224619b45b4f90263213f50f5266d64d7e45590ee95ecc0f8709780d96f2c
  Stored in directory: /tmp/pip-ephem-wheel-cache-brxiielz/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers


In [5]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-2pl7tiva
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-2pl7tiva
  Resolved https://github.com/huggingface/transformers.git to commit c4e20698985887215f7e91a02621265f047af2d7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 558.8/558.8 kB 31.3 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.55.0.dev0-py3-none-any.whl size=12117010 sha256=e0162cc1394a23262750abc54556b121e1400a2c7dfc3f507ccfa175a7d13e26
  Stored in directory: /tmp/pip-ephem-wheel-cache-y4jcojxn/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.33.5
    Uninstalling huggingface-

In [1]:
!pip install --upgrade torch torchvision accelerate

In [2]:
from transformers import AutoModelForCausalLM, AutoProcessor
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

model = AutoModelForCausalLM.from_pretrained(
    "nvidia/audio-flamingo-3",
    trust_remote_code=True,
    torch_dtype=torch.float16
).to(device)

processor = AutoProcessor.from_pretrained("nvidia/audio-flamingo-3", trust_remote_code=True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


ValueError: The checkpoint you are trying to load has model type `llava_llama` but Transformers does not recognize this architecture. This could be because of an issue with the checkpoint, or because your version of Transformers is out of date.

You can update Transformers with the command `pip install --upgrade transformers`. If this does not work, and the checkpoint is very new, then there may not be a release version that supports this model yet. In this case, you can get the most up-to-date code by installing Transformers from source with the command `pip install git+https://github.com/huggingface/transformers.git`

In [1]:
!pip install --upgrade torch torchvision accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 117.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 97.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2